In [2]:
## Install Selenium so we can use its libraries later
!pip install selenium
!pip install selenium-wire
!pip install python-dotenv

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/f9/2f/9c6eef6487faca5006ae1ba43cf6ab627c7e3d2a7ec5a3b8728e2105472d/selenium-4.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/a3/dd/b61fa61b186d3267ef3903048fbee29132963ae762fb70b08d4a3cd6f7aa/trio-0.22.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/f2/25/868488d8b05104e3d1bf162565bbadd5fe6d62bf903710e6402c3aaceb1e/trio_websocket-0.10.4-py3-none-any.whl.metadata
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Obtaining dependency information for exceptiongroup from https://files.pythonhosted.org/packages/ad/83/b71e58666f156a39fb29417e4c8ca4bc7400c0dd4ed9e8842ab54dc8c344/exceptiongroup-1.1.3-py3-none-any.whl.metadata
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  

In [5]:
## Bring in the required libraries
import pandas as pd
import numpy as np
import time
import math
import seleniumwire
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from seleniumwire import webdriver
from seleniumwire.utils import decode

## PART 1 - take raw data from Neufast, then extra data & transform into the format we need for ISS entry
### There are 6 Steps in Part I - run the following cells one by one, in order

### STEP 1 - *** Replace the path below with the location of the excel file ***

In [21]:
# take the raw excel
# file path: C:\Users\hyyiu\Desktop\testing\raw data from neufast\Candidate Report_no_mandarin.xlsx
# *** STEP 1 ***
raw = pd.read_excel(r'C:\Users\hyyiu\Desktop\AI_interview\raw data from AI interview platform\demo.xlsx', 
                    sheet_name=['Candidate Info', 'Auto-Score', 'Candidate Answer'])
sid_df = raw.get('Candidate Info')
score_df = raw.get('Auto-Score')
preference_df = raw.get('Candidate Answer')

print('Step 1 --- Successful!')

Step 1 --- Successful!


### STEP 2

In [22]:
# Get all Student IDs & Student Names from dataframe 1 ('Candidate Info' Sheet)

sid_col = sid_df['University']
student_name_col = sid_df['Fullname']

print('Step 2 --- Successful!')

Step 2 --- Successful!


### Step 3 - convert scores from Neufast 10-point scale to ISS 3-point scale

In [23]:
## This is a function to convert a SINGLE SCORE from a 10-point scale to ISS 3-point scale
def conversion(y):
    result_map = {
        (0, 0.99): 1,
        (1, 1.5): 1.5,
        (1.51, 2): 2,
        (2.01, 2.5): 2.5,
        (2.51, 3): 3,
        (3.01, 100): 3,
    }
    result = y/10*3
    
    for range_min, range_max in result_map:
        if result >= range_min and result <= range_max:
            return result_map[(range_min, range_max)]
    return math.floor(result)

In [24]:
## Converter Factory 

def converter(col):
    for index, value in col.items():
        col.loc[index] = conversion(value)
    
    return col

In [25]:
## Clean all score columns to make sure there is no empty cell. If so, fill it with 0

communication_col = score_df['Impression Score'].fillna(0)
teamwork_col = score_df['Contextual Performance Score'].fillna(0)
cultural_col = score_df['Adaptive Performance Score'].fillna(0)
independence_col = score_df['Task Performance Score'].fillna(0)
resilience_col = score_df['Overall Score'].fillna(0)

In [26]:
## Now convert all 5 columns

communication_col = converter(communication_col)
teamwork_col = converter(teamwork_col)
cultural_col = converter(cultural_col)
independence_col = converter(independence_col)
resilience_col = converter(resilience_col)

### STEP 3 - Sort out English & Mandarin scores (fill Mandarin score with 0 for non-Mandarin speakers)

In [27]:
# Round down overall score and calculate English score

comments_col = score_df['Overall Score'].round(decimals=2)
english_col = score_df['English Oral Communication'].divide(2).round(decimals=0)
 
# if there is no column for Mandarin, create a new one and fill it with zero
if 'Mandarin Oral Communication' not in score_df.columns:
    score_df['Mandarin Oral Communication'] = 0
    putonghua_col = score_df['Mandarin Oral Communication']
else:
    putonghua_col = score_df['Mandarin Oral Communication'].divide(2).round(decimals=0)
    
print('Step 3 --- Successful!')

Step 3 --- Successful!


#### STEP 4

In [28]:
# take columns from preference_df ('Candidate Answer' Sheet)

preferred_i_locations_col = preference_df['Q5: (International WIE) Please put up to 3 preferred locations in the answer box. NA if not interested. (E.g.: Thailand, Singapore, Korea) (Short Question)']
preferred_i_budget_col = preference_df['Q7: (International WIE) Please state your budget in HKD here. Put NA if not applicable. (E.g.: HKD30000) (Short Question)']
preferred_m_locations_col = preference_df['Q6: (Mainland WIE) Please put up to 3 preferred locations in the answer box. NA if not interested. (E.g.: Shenzhen, Shanghai, Hangzhou) (Short Question)']
preferred_m_budget_col = preference_df['Q8: (Mainland WIE) Please state your budget in HKD here. Put NA if not applicable. (E.g.: HKD30000) (Short Question)']

print('Step 4 --- Successful!')

Step 4 --- Successful!


### STEP 5 - make sure human input has no whitespace around it

In [29]:
# Strip extra space around all text input & column names first
# columns with manual input: sid_col', student_name_col, preferred_i_locations_col, 
# preferred_i_budget_col, preferred_m_locations_col, preferred_m_budget_col

sid_col = sid_col.str.strip()
student_name_col = student_name_col.str.strip()
preferred_i_locations_col = preferred_i_locations_col.str.strip()
preferred_i_budget_col = preferred_i_budget_col.str.strip()
preferred_m_locations_col = preferred_m_locations_col.str.strip()
preferred_m_budget_col = preferred_m_budget_col.str.strip()

print('Step 5 --- Successful!')

Step 5 --- Successful!


### STEP 6 - Bring all columns together & rename them for our new Excel

In [30]:
## Bringing all columns together:
data = [sid_col, student_name_col, communication_col, teamwork_col, cultural_col, independence_col, resilience_col,
                comments_col, english_col, putonghua_col, preferred_i_locations_col, preferred_i_budget_col, preferred_m_locations_col,
                preferred_m_budget_col]

# Create a new dataframe with all column
column_names = ['studentID','studentName', 'communication', 'teamwork', 'culturalAwareness', 'independence',
               'resilience', 'comments', 'english', 'putonghua', 'internationalLocations', 'intlBudget', 'mainlandLocations', 'mainlandBudget']
combined_df = pd.concat(data, axis=1)
combined_df.columns = column_names
combined_df.head(5)

print('Step 6 --- Successful!')

Step 6 --- Successful!


### STEP 7 - All 'AI Interview' and the student iss link column to the new Excel

In [8]:
# bring in OS & dotenv library to load environment variables

import os
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
# os.environ['BASE_URL'] = 'base_url'

## Add the Interviewer column 
combined_df['interviewedBy'] = 'AI Interview'

## Student ISS Link colum to the dataframe
base_url = os.environ.get('BASE_URL')
combined_df['student_link'] = base_url+combined_df['studentID']

# Make student ID the index of each row, and view new dataframe again
final_df = combined_df.set_index('studentID')
# FINALLY, save the merged dataframe as a new excel file
# MAKE SURE THERE IS NO FILE THAT HAS THE SAME FILE NAME IN THE FOLDER
final_df.to_excel('scores_for_iss_entry.xlsx')

print('Step 7 --- Successful!')
print('Now, go to the folder and check if there is a file called "scores_for_iss_entry"!')

Step 7 --- Successful!
Now, go to the folder and check if there is a file called "scores_for_iss_entry"!


## PART 2 - Auto score entry on ISS

### STEP 1 - open browser, sign into ISS ***please update the NetID & Password below***

In [32]:
## Sign into ISS
# default to open browser with chrome & go to ISS
browser = webdriver.Chrome()
browser.get(os.environ.get('ISS'))
time.sleep(2)
# Sign in
browser.find_element(By.NAME, 'j_username').send_keys(os.environ.get('NETID')) # enter your NetID here
browser.find_element(By.NAME, 'j_password').send_keys(os.environ.get('PASSWORD')) # enter your password here
browser.find_element(By.NAME, 'buttonAction').click()


### STEP 2 - If sign-in is successful, run the following code to automate the data entry process

In [33]:
def enter_all_students():
    ## Make sure excel is in the excel same format with the column names and fields
    ## Make sure the columns are in the same order as the fields being entered on ISS
    ## open excel file. Make sure the excel location is in the same folder where this Notebook file is
    scores = pd.read_excel('scores_for_iss_entry.xlsx', index_col=0)
    # check the first 5 rows to visualise the rows & columns
    scores.head(5)
    # Loop starts here
    for index, row in scores.iterrows():
#         print(f'Working on Student ID: {index} .....')
        try: 
            # go to the current student's page on ISS
            browser.get(row['student_link'])
            time.sleep(1)
            browser.find_element(By.NAME, 'communication').clear()
            browser.find_element(By.NAME, 'communication').send_keys(row['communication'])
            # Enter score for "Team work"
            browser.find_element(By.NAME, 'jobSpecification').clear()
            browser.find_element(By.NAME, 'jobSpecification').send_keys(row['teamwork'])
            # Enter score for "Cultural Awareness"
            browser.find_element(By.NAME, 'culturalAwareness').clear()
            browser.find_element(By.NAME, 'culturalAwareness').send_keys(row['culturalAwareness'])
            # Enter score for "Independence & Problem Solving"
            browser.find_element(By.NAME, 'independence').clear()
            browser.find_element(By.NAME, 'independence').send_keys(row['independence'])
            # Enter score for "Resilience"
            browser.find_element(By.NAME, 'teamwork').clear()
            browser.find_element(By.NAME, 'teamwork').send_keys(row['resilience'])
            #Enter score for English Proficiency
            browser.find_element(By.NAME, 'english').clear()
            browser.find_element(By.NAME, 'english').send_keys(row['english'])
            #Enter score for Putonghua Proficiency
            # check if the n/a box is checked 
            if browser.find_element(By.NAME, 'putonghuaNa').is_selected():
                continue 
            else:
                browser.find_element(By.NAME, 'putonghua').clear()
                # if not checked, check if row['putonghua'] == 0,
                if (row['putonghua'] == 0): 
                # then check the n/a box
                    browser.find_element(By.NAME, 'putonghuaNa').click()
                else: 
                    # if not 0 or checked => enter the score
                    browser.find_element(By.NAME, 'putonghua').send_keys(row['putonghua'])
            #Enter "Comments"
            browser.find_element(By.NAME, 'comments').clear()
#             browser.find_element(By.NAME, 'comments').send_keys('Overall Score from Neufast: ')
            browser.find_element(By.NAME, 'comments').send_keys(row['comments'])
            # Enter Mainland WIE
            browser.find_element(By.NAME, 'recommenedProgramme1').clear()
            browser.find_element(By.NAME, 'recommenedProgramme1').send_keys(row['mainlandLocations'])
            # Enter budget for Mainland WIE 
            browser.find_element(By.NAME, 'recommenedProgramme2').clear()
            browser.find_element(By.NAME, 'recommenedProgramme2').send_keys(row['mainlandBudget'])
            # Enter International WIE locations
            browser.find_element(By.NAME, 'recommenedProgramme3').clear()
            browser.find_element(By.NAME, 'recommenedProgramme3').send_keys(row['internationalLocations'])
            # Enter International WIE locations
            browser.find_element(By.NAME, 'recommenedProgramme4').clear()
            browser.find_element(By.NAME, 'recommenedProgramme4').send_keys(row['intlBudget'])
            #Enter "Interviewed By"
            browser.find_element(By.NAME, 'interviewedBy').clear()
            browser.find_element(By.NAME, 'interviewedBy').send_keys(row['interviewedBy'])
            # Hit "Save" button
            browser.find_element(By.NAME, 'saveButton').click() ## if this is successful, print SAVED!
            time.sleep(1)
            if (str(browser.current_url) == str(row['student_link'])):
                print(f'WARNING! Student ID: {index} is not SAVED! Please check input values.')
            else:
                print(f'---- Student ID: {index} is DONE ---- ') 
        except:
            continue
    print('--- End of Programme ---')
     
enter_all_students()

---- Student ID: 20073362D is DONE ---- 
---- Student ID: 22028032D is DONE ---- 
---- Student ID: 22031872D is DONE ---- 
---- Student ID: 19049672D is DONE ---- 
---- Student ID: 20039693D is DONE ---- 
--- End of Programme ---
